In [1]:
! pip install albumentations
! pip install pretrainedmodels

     |████████████████████████████████| 58 kB 434 kB/s eta 0:00:011
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=464bcad82e1e628b84bb3769d64fe32d1b332c4c0b5ce46632148af719fcb482
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


## 一、导入库

In [2]:
from PIL import Image
from torchvision import models
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from collections import Counter
from albumentations.pytorch import ToTensorV2
from albumentations import FancyPCA
import albumentations as A
import pretrainedmodels
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.utils.data as data
import cv2
import os
import random
import pandas as pd
import json

# 设置随机种子
seed = 2020
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)            # 为CPU设置随机种子
torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)   # 为所有GPU设置随机种子
os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现。

## 二、获取路径以及类别,并划分训练验证集

In [3]:
DIR_INPUT = '/kaggle/input/plant-pathology-2020-fgvc7/images/'
data_df = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
test_df = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
data_paths = [DIR_INPUT + id for id in data_df['image_id']]
test_paths = [DIR_INPUT + id for id in test_df['image_id']]
data_labels = []
for i in range(len(data_df)):
    label = data_df.loc[i, ['healthy', 'multiple_diseases', 'rust', 'scab']].values
    data_labels.append(int(np.argwhere(label==1)))

# 打散
data_paths = np.array(data_paths)
data_labels = np.array(data_labels)
p = np.random.permutation(len(data_labels))
data_paths = data_paths[p]
data_labels = data_labels[p]

# 划分训练验证集
split_k = len(data_paths) // 8
train_paths = data_paths[split_k:]
train_labels = data_labels[split_k:]
val_paths = data_paths[:split_k]
val_labels = data_labels[:split_k]


1. ## 三、制作data loader类

In [4]:
# data loader
class ImageDataset(data.Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.paths[index]
        label = self.labels[index]
        image = cv2.imread(img_path + ".jpg")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        return image, int(label)

    def __len__(self):
        return len(self.paths)

# test loader
class TestDataset(data.Dataset):
    def __init__(self, paths, transform=None):
        self.paths = paths
        self.transform = transform

    def __getitem__(self, index):
        img_path = self.paths[index]
        image = cv2.imread(img_path + ".jpg")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            res = self.transform(image=image)
            image = res['image']
        return image

    def __len__(self):
        return len(self.paths)

In [19]:
train_transform = A.Compose([
            A.Resize(height=512, width=512),
            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
                A.RandomBrightness(limit=0.1, p=0.5),
            ], p=1),
            A.GaussNoise(),
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(rotate_limit=1, p=0.5),
            FancyPCA(alpha=0.1, p=0.5),
            # blur
            A.OneOf([
                A.MotionBlur(blur_limit=3), A.MedianBlur(blur_limit=3), A.GaussianBlur(blur_limit=3),
            ], p=0.5),
            # Pixels
            A.OneOf([
                A.IAAEmboss(p=0.5),
                A.IAASharpen(p=0.5),
            ], p=1),
            # Affine
            A.OneOf([
                A.ElasticTransform(p=0.5),
                A.IAAPiecewiseAffine(p=0.5),
            ], p=1),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ])

valid_transform = A.Compose([
            A.Resize(height=512, width=512),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ])


test_transform = A.Compose([
            A.Resize(height=512, width=512),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ])

trainset = ImageDataset(train_paths, train_labels, train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
validset = ImageDataset(val_paths, val_labels, valid_transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=False, num_workers=4)
testset = TestDataset(test_paths, test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=4)

## 四、构建模型

In [6]:
class Net(nn.Module):
    def __init__(self, alexnet):
        super(Net, self).__init__()
        self.backbone = alexnet._features
        self.fc = nn.Linear(1024, 4)

    def forward(self, x):
        batch_size, C, H, W = x.shape
        x = self.backbone(x)
        x = F.adaptive_avg_pool2d(x, 2).reshape(batch_size, -1)
        x = self.fc(x)
        return x

In [7]:
# 根据环境自动选择是否使用GPU
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
# 使用Alexnet网络进行图像分类
alexnet = pretrainedmodels.__dict__['alexnet'](num_classes=1000, pretrained='imagenet')
# 将原始alexnet送入自定义的类中
alexnet = Net(alexnet)

alexnet.to(device)
# 打印alexnet网络结构
print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth



Net(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1024, out_features=4, bias=True)
)


## 五、配置训练参数

In [8]:
# 使用交叉熵损失进行训练
criterion = nn.CrossEntropyLoss()
# 使用随机梯度下降法进行优化模型，并且设置学习率为0.001
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)
# 训练2个epoch
EPOCH = 2

## 六、训练

In [9]:
for epoch in range(EPOCH):
    running_loss = 0
    train_correct = 0
    train_total = 0
    for i, data in enumerate(trainloader):
        images, labels = data[0].to(device), data[1].to(device, dtype=torch.int64)
        optimizer.zero_grad()
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/20))
            running_loss = 0.0
    train_accuracy = train_correct / train_total
    print('train dataset accuracy %.4f' % train_accuracy)
    
    test_correct = 0
    test_total = 0
    res = []
    with torch.no_grad():
        for data in validloader:
            images, labels = data[0].to(device), data[1].to(device, dtype=torch.int64)
            outputs = alexnet(images)
            _, predicted = torch.max(outputs.data, 1)
            for p in predicted:
                res.append(int(p))

    print("Val Accuracy: ", np.sum(np.array(res)==np.array(val_labels)) / len(val_labels))

[1,    20] loss: 1.213
[1,    40] loss: 1.112
[1,    60] loss: 0.890
[1,    80] loss: 0.608
[1,   100] loss: 0.635
train dataset accuracy 0.6305
Val Accuracy:  0.8325991189427313
[2,    20] loss: 0.592
[2,    40] loss: 0.535
[2,    60] loss: 0.519
[2,    80] loss: 0.668
[2,   100] loss: 0.381
train dataset accuracy 0.8156
Val Accuracy:  0.8502202643171806


## 七、生成提交文件

In [21]:
test_preds = None
submission_df = pd.read_csv('../input/plant-pathology-2020-fgvc7/sample_submission.csv')
with torch.no_grad():
    for data in testloader:
        images = data.to(device)
        outputs = alexnet(images)

        if test_preds is None:
            test_preds = outputs.data.cpu()
        else:
            test_preds = torch.cat((test_preds, outputs.data.cpu()), dim=0)

submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = torch.softmax(test_preds, dim=1)
submission_df.to_csv('submission.csv', index=False)

ValueError: not enough values to unpack (expected 4, got 3)